In [4]:
import pandas as pd
import numpy as np
import os

In [6]:
folder_path = '../../Data/sclc_ucologne_2015/unused_data'
file_name = 'data_clinical_patient.txt'
file_path = folder_path + '/' + file_name
# Load the RNAseq data from the csv file
data = pd.read_csv(file_path, sep='\t', index_col=0, header=0, usecols=['PATIENT_ID','AGE', 'ETHNICITY', 'SEX', 'OS_STATUS', 'OS_MONTHS']  )


# the columns correspond to the patient IDs
# delete rows with no OS_STATUS
data = data[data['OS_STATUS'].notna()]
# Get the column names
data.head()

ValueError: Usecols do not match columns, columns expected but not found: ['ETHNICITY', 'OS_MONTHS', 'SEX', 'AGE', 'OS_STATUS']

In [ ]:
data["race"] = data["ETHNICITY"]
data['year_of_diagnosis'] = data['AGE']
data['year_of_birth'] = 2015 - data['AGE'] 
data['overall_survival'] = data['OS_MONTHS']
data['vital_status'] = data['OS_STATUS']
data['disease_specific_survival'] = data['OS_MONTHS']
data['primary_site'] = 'Bronchus and lung'


In [ ]:
#replace "Asian" with "NOT HISPANIC OR LATINO, other with nan
data["ETHNICITY"].replace({"Asian": "NOT HISPANIC OR LATINO"}, inplace=True)
#other with Nan

# Replace all other values that are not "NOT HISPANIC OR LATINO" with NaN
data["ETHNICITY"] = data["ETHNICITY"].apply(lambda x: "NOT HISPANIC OR LATINO" if x == "NOT HISPANIC OR LATINO" else np.nan)



In [ ]:
data["race"].replace({"Caucasian": "WHITE", "Asian": "ASIAN"}, inplace=True)

In [ ]:
#change column names
data.rename(columns = {'PATIENT_ID':'patient_id', 'SEX': 'gender', 'ETHNICITY':'ethnicity' }, inplace = True)

In [ ]:
#drop AGE, OS_STATUS, OS_MONTHS
data.drop(columns = ['AGE', 'OS_STATUS', 'OS_MONTHS'], inplace = True)

In [ ]:
data.head()

,gender,ethnicity,race,year_of_diagnosis,year_of_birth,overall_survival,vital_status,disease_specific_survival,primary_site
PATIENT_ID,,,,,,,,,
sclc_ucologne_2015_S00022,Male,NaN,NaN,47.0,1968.0,38.0,1:DECEASED,38.0,Bronchus and lung
sclc_ucologne_2015_S00035,Female,NaN,NaN,65.0,1950.0,12.0,1:DECEASED,12.0,Bronchus and lung
sclc_ucologne_2015_S00050,Male,NaN,NaN,47.0,1968.0,42.0,1:DECEASED,42.0,Bronchus and lung
sclc_ucologne_2015_S00213,Male,NaN,NaN,65.0,1950.0,13.0,1:DECEASED,13.0,Bronchus and lung
sclc_ucologne_2015_S00356,Female,NaN,NaN,54.0,1961.0,33.0,1:DECEASED,33.0,Bronchus and lung


In [ ]:
#save to tsv
#data.to_csv(folder_path + '/data_clinical_patient.tsv', sep='\t')


In [ ]:
#smaller dataset for testing, with less columns (patients) and rows (genes)

data = data.iloc[:5, :5]
data.to_csv(folder_path + '/data_clinical_patient_small.csv', sep='\t')


NameError: name 'data' is not defined